# **FIRST ATTEMPT AFTER WATCHING THE TUTORIAL**

In [4]:
import numpy as np
from PIL import Image

import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim


import torchvision
import torchvision.transforms as trans

In [5]:
transformed = trans.Compose([
    trans.ToTensor(),
    trans.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))
])

In [9]:
load_data = torchvision.datasets.CIFAR10(root='data', download=True)

100%|██████████| 170M/170M [00:01<00:00, 103MB/s]


AttributeError: 'CIFAR10' object has no attribute 'load_data'